![databricks_academy_logo.png](../Includes/images/databricks_academy_logo.png "databricks_academy_logo.png")

# Transforming Data Using the Medallion Architecture
![get-started-de-medallion.png](../Includes/images/get-started-de-medallion.png "get-started-de-medallion.png")

## Important: Select Environment 4
The cells below may not work in other environments. To choose environment 4: 
1. Click the ![environment.png](../Includes/images/environment.png "environment.png") button on the right sidebar
1. Open the **Environment version** dropdown
1. Select **4**

## Classroom Setup

Run the following cell to configure your working environment for this lesson.

In [0]:
####################################################################################
# Set python variables for catalog, schema, and volume names (change, if desired)
catalog_name = "dbacademy"
schema_name = "transforming_data"
volume_name = "myfiles"
####################################################################################

####################################################################################
# Create the catalog, schema, and volume if they don't exist already
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.{volume_name}")
####################################################################################

####################################################################################
# Creates a file called employees.csv in the specified catalog.schema.volume
import pandas as pd
data = [
    ["1111", "Kristi", "USA", "Manager"],
    ["2222", "Sophia", "Greece", "Developer"],
    ["3333", "Peter", "USA", "Developer"],
    ["4444", "Zebi", "Pakistan", "Administrator"]
]
columns = ["ID", "Firstname", "Country", "Role"] 
df = pd.DataFrame(data, columns=columns)
file_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/employees.csv"
df.to_csv(file_path, index=False)
################################################################################

####################################################################################
# Creates a file called employees2.csv in the specified catalog.schema.volume
data = [
    [5555, 'Alex','USA', 'Instructor'],
    [6666, 'Sanjay','India', 'Instructor']
]
columns = ["ID","Firstname", "Country", "Role"]

## Create the DataFrame
df = pd.DataFrame(data, columns=columns)

## Create the CSV file in the course Catalog.Schema.Volume
df.to_csv(f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/employees2.csv", index=False)
####################################################################################

## A. Configure and Explore Your Environment

1. Set the default catalog and schema.

In [0]:
## Set the default catalog and schema
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

DataFrame[]

2. View the available files in your schema's **myfiles** volume. Confirm that the volume contains two CSV files, **employees.csv** and **employees2.csv**.

In [0]:
spark.sql(f"LIST '/Volumes/{catalog_name}/{schema_name}/{volume_name}/' ").display()

path,name,size,modification_time
/Volumes/dbacademy/transforming_data/myfiles/employees.csv,employees.csv,137,1771521796000
/Volumes/dbacademy/transforming_data/myfiles/employees2.csv,employees2.csv,80,1771521796000


## B. Simple Example of the Medallion Architecture

### BRONZE
**Objective:** Create a table using all of the CSV files in the **myfiles** volume.

1. Execute the cell to perform the following steps:

    - The `DROP TABLE IF EXISTS` statement drops the **current_employees_bronze** table if it already exists (for demonstration purposes).
    
    - The `CREATE TABLE IF NOT EXISTS` statement creates the Delta table **current_employees_bronze** if it doesn't already exist and defines its columns.
    
    - The `COPY INTO` statement:
        - Loads all the CSV files from the **myfiles** volume in the schema into the **current_employees_bronze** table.
        - Uses the first row as headers and infers the schema from the CSV files.
    
    - The final `SELECT` query displays all rows from the **current_employees_bronze** table.


View the results and confirm that the table contains **6 rows** and **4 columns**.


In [0]:
%sql
-- Drop the table if it exists for demonstration purposes
DROP TABLE IF EXISTS current_employees_bronze;

-- Create an empty table and columns
CREATE TABLE IF NOT EXISTS current_employees_bronze (
  ID INT,
  FirstName STRING,
  Country STRING,
  Role STRING
  );

In [0]:
## Create the bronze raw ingestion table and include the CSV file name for the rows
spark.sql(f'''
  COPY INTO current_employees_bronze
  FROM '/Volumes/{catalog_name}/{schema_name}/{volume_name}/'
  FILEFORMAT = CSV
  FORMAT_OPTIONS (
    'header' = 'true', 
    'inferSchema' = 'true'
)
''').display()

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
6,6,0


In [0]:
%sql
SELECT * 
FROM current_employees_bronze;

ID,FirstName,Country,Role
1111,Kristi,USA,Manager
2222,Sophia,Greece,Developer
3333,Peter,USA,Developer
4444,Zebi,Pakistan,Administrator
5555,Alex,USA,Instructor
6666,Sanjay,India,Instructor


### SILVER
**Objective**: Transform the bronze table and create the silver table.

1. Create a table named **current_employees_silver** from the **current_employees_bronze** table. 

    The table will:
    - Select the columns **ID**, **FirstName**, **Country**.
    - Convert the **Role** column to uppercase.
    - Add two new columns: **CurrentTimeStamp** and **CurrentDate**.

In [0]:
%sql
-- Create a temporary view to use to merge the data into the final silver table
CREATE OR REPLACE TABLE current_employees_silver AS 
SELECT 
  ID,
  FirstName,
  Country,
  upper(Role) as Role,                 -- Upcase the Role column
  current_timestamp() as CurrentTimeStamp,    -- Get the current datetime
  date(CurrentTimeStamp) as CurrentDate       -- Get the date
FROM current_employees_bronze;

num_affected_rows,num_inserted_rows


2. View the **current_employees_silver** table. Confirm that the table contains 6 rows and 6 columns.

In [0]:
%sql
SELECT * 
FROM current_employees_silver;

ID,FirstName,Country,Role,CurrentTimeStamp,CurrentDate
1111,Kristi,USA,MANAGER,2026-02-19T17:27:16.246Z,2026-02-19
2222,Sophia,Greece,DEVELOPER,2026-02-19T17:27:16.246Z,2026-02-19
3333,Peter,USA,DEVELOPER,2026-02-19T17:27:16.246Z,2026-02-19
4444,Zebi,Pakistan,ADMINISTRATOR,2026-02-19T17:27:16.246Z,2026-02-19
5555,Alex,USA,INSTRUCTOR,2026-02-19T17:27:16.246Z,2026-02-19
6666,Sanjay,India,INSTRUCTOR,2026-02-19T17:27:16.246Z,2026-02-19


### GOLD
**Objective:** Aggregate the silver table to create the final gold table.

1. Create a temporary view named **temp_view_total_roles** that aggregates the total number of employees by role. Then, display the results of the view.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW temp_view_total_roles AS 
SELECT
  Role, 
  count(*) as TotalEmployees
FROM current_employees_silver
GROUP BY Role;


SELECT *
FROM temp_view_total_roles;

Role,TotalEmployees
MANAGER,1
DEVELOPER,2
ADMINISTRATOR,1
INSTRUCTOR,2


2. Create the final gold table named **total_roles_gold** with the specified columns.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS total_roles_gold (
  Role STRING,
  TotalEmployees INT
);

3. Insert all rows from the aggregated temporary view **temp_view_total_rows** into the **total_roles_gold** table, overwriting the existing data in the table. This overwrites the data in a table but keeps the existing schema and table definition and properties.

    Confirm the following:
    - **num_affected_rows** is *4*
    - **num_inserted_rows** is *4*

In [0]:
%sql
INSERT OVERWRITE TABLE total_roles_gold
SELECT * 
FROM temp_view_total_roles;

num_affected_rows,num_inserted_rows
4,4


4. Query the **total_roles_gold** table to view the total number of employees by role.

In [0]:
%sql
SELECT *
FROM total_roles_gold;

Role,TotalEmployees
INSTRUCTOR,2
ADMINISTRATOR,1
DEVELOPER,2
MANAGER,1


5. View the history of the **total_roles_gold** table.

In [0]:
%sql
DESCRIBE HISTORY total_roles_gold;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,queryHistoryStatementId,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2026-02-19T17:30:44.000Z,74576766048446,pavanikorada2004@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> true, partitionBy -> [])",null,List(2273195081328557),9d150b5d-5caf-41c8-88b2-593518c3db13,0219-172245-41wiu8hr-v2n,0,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 4, numOutputBytes -> 959)",null,Databricks-Runtime/18.0.x-aarch64-photon-scala2.13
0,2026-02-19T17:30:29.000Z,74576766048446,pavanikorada2004@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true"",""delta.enableRowTracking"":""true"",""delta.rowTracking.materializedRowCommitVersionColumnName"":""_row-commit-version-col-98866793-716e-4c4a-b775-8dceee0ff681"",""delta.rowTracking.materializedRowIdColumnName"":""_row-id-col-57722095-67c3-43f1-9204-0c6e113c66b7""}, statsOnLoad -> false)",null,List(2273195081328557),0dad30ff-4f09-4609-abd6-6d11475e7350,0219-172245-41wiu8hr-v2n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/18.0.x-aarch64-photon-scala2.13


## C. Data Governance and Security
**Objectives:** View the lineage of the **total_roles_gold** table and learn how to set its permissions.

### 1. Complete the following to open the schema in the **Catalog Explorer**.

a. Select the Catalog icon ![catalog_icon](../Includes/images/catalog_icon.png) in the left navigation bar. 

c. Select the refresh icon ![refresh_icon](../Includes/images/refresh_icon.png) to refresh the catalog.

d. Expand the catalog (default is **dbacademy**). Within the catalog, you should see a few schemas (databases).

e. Find and select the **transforming_data** schema.

f.  Click the options icon ![options_icon](../Includes/images/options_icon.png) to the right of the schema name and choose **Open in Catalog Explorer**.

g. Notice that the three tables we created in the demo: **current_employees_bronze**, **current_employees_silver**, and **total_roles_gold** are shown in the **Catalog Explorer** for your schema.

h. In the **Catalog Explorer** select the **total_roles_gold** table.

Leave the **Catalog Explorer** tab open.

### 2. Complete the following to view the **total_roles_gold** table's permissions, history, lineage and insights in Catalog Explorer: 

   a. **Permissions**. 

  - Select the **Permissions** tab. This will display all permissions on the table. Currently the table does not have any permissions set.

  - Select **Grant**. This allows you to add multiple principals and assign privileges to them. Users must have access to the Catalog and Schema of the table.
  
  - Select **Cancel**. 

   b. **History**

  - Select the **History** tab. If needed, click **Select compute** and choose the default compute from the dropdown. This will display the table's history. The **total_roles_gold** table currently has two versions. 

   c. **Lineage**

  - Select the **Lineage** tab. This displays the table's lineage. Confirm that the **current_employees_silver** table is shown.

  - Select the **See lineage graph** button ![see_lineage_graph_button](../Includes/images/see_lineage_graph_button.png). This displays the table's lineage visually. You can select the ![plus_button](../Includes/images/plus_button.png) icon to zoom in.

  - Close out of the lineage graph.

   d. **Insights**

  - Select the **Insights** tab. You can use the Insights tab in **Catalog Explorer** to view the most frequent recent queries and users of any table registered in Unity Catalog. The Insights tab reports on frequent queries and user access for the past 30 days.
  
   e. Close the **Catalog Explorer** browser tab. 

##D. Cleanup
1. Drop the **transforming_data** schema.

In [0]:
%sql
DROP SCHEMA IF EXISTS transforming_data CASCADE;